In [ ]:
import os
import shutil
import cv2
import keras
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input, MobileNetV2
import pydicom
from pathlib import Path
from tqdm.auto import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import skimage.io
import tqdm
import glob
import tensorflow
import matplotlib.image as mpimage
import matplotlib.pyplot as plt
%matplotlib inline
from PIL import Image
import random
import gc
import re
import cv2
from tqdm import tqdm
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.layers import Activation, Dropout, Flatten, BatchNormalization, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint


# import pandas as pd
# import tensorflow as tf
# from sklearn.preprocessing import OneHotEncoder
# from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
# from sklearn.model_selection import train_test_split
# import yaml
# from kaggle_secrets import UserSecretsClient
# import cv2
# graphing
# TF model stuff
# from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
# from sklearn.utils.class_weight import compute_class_weight
# from sklearn.metrics import accuracy_score, roc_auc_score, classification_report
# from tensorflow.keras.preprocessing.image import load_img, img_to_array


In [ ]:
SIIM_COVID19_DETECTION_DIR = '/kaggle/input/siim-covid19-detection/'
PART0_RESIZED_DIR = '../input/siim-covid19-resized-to-512px-jpg'


TEMP_DIR = '/kaggle/temp/'

INPUT_DIR = PART0_RESIZED_DIR+'/train/'

OUTPUT_DIR = DATASET_DIR = TEMP_DIR+'/train/'
TRAIN_DIR = DATASET_DIR + 'train/'
TA_DIR = TRAIN_DIR+'ta/'
IA_DIR = TRAIN_DIR+'ia/'
AA_DIR = TRAIN_DIR+'aa/'
NP_DIR = TRAIN_DIR+'np/'

# WORKING_DIR = '/kaggle/working/'

WANDB_PROJECT_NAME = 'project8-kaggle-covid19'
WANDB_ENTITY_NAME = ''

TRAIN_IMAGE_LEVEL_PATH = SIIM_COVID19_DETECTION_DIR+'train_image_level.csv'
TRAIN_STUDY_LEVEL_PATH = SIIM_COVID19_DETECTION_DIR+'train_study_level.csv'
META_PATH = PART0_RESIZED_DIR+'meta.csv'

BATCH_SIZE = 32
EPOCHS = 25
IMG_SIZE = WIDTH = HEIGHT = 224
LEARNING_RATE = 0.00008

# INTERPOLATION = cv2.INTER_LANCZOS4


In [ ]:
df_train_image_level = pd.read_csv(TRAIN_IMAGE_LEVEL_PATH)
df_train_study_level = pd.read_csv(TRAIN_STUDY_LEVEL_PATH)

df_train_image_level['id'] = df_train_image_level.apply(lambda row: row.id.split('_')[0], axis=1)
df_train_image_level['path'] = df_train_image_level.apply(lambda row: INPUT_DIR+row.id+'.jpg', axis=1)
df_train_image_level['image_level'] = df_train_image_level.apply(lambda row: row.label.split(' ')[0], axis=1)

df_train_study_level['id'] = df_train_study_level.apply(lambda row: row.id.split('_')[0], axis=1)
df_train_study_level.columns = ['StudyInstanceUID', 'Negative for Pneumonia', 'Typical Appearance', 'Indeterminate Appearance', 'Atypical Appearance']

df_train_study_level.head()

In [ ]:
df_train_image_level.head()

In [ ]:
df_train_image_level = df_train_image_level.merge(df_train_study_level, on='StudyInstanceUID',how="left")
df_train_image_level = df_train_image_level[['id','StudyInstanceUID','path','Negative for Pneumonia','Typical Appearance','Indeterminate Appearance','Atypical Appearance']]
df_train_image_level = df_train_image_level.dropna()
df_train_image_level = df_train_image_level[~df_train_image_level.duplicated(subset=['StudyInstanceUID'], keep='first')]
df_train_image_level = df_train_image_level.reset_index(drop=True)
df_train_image_level.head()

In [ ]:
[os.makedirs(dir, exist_ok=True) for dir in [TA_DIR,IA_DIR,AA_DIR,NP_DIR]]
for i in tqdm(range(len(df_train_image_level))):
    row = df_train_image_level.loc[i]
    if row['Typical Appearance']:
        shutil.copy(row.path, f'{TA_DIR}{row.id}.jpg')
    if row['Indeterminate Appearance']:
        shutil.copy(row.path, f'{IA_DIR}{row.id}.jpg')
    if row['Atypical Appearance']:
        shutil.copy(row.path, f'{AA_DIR}{row.id}.jpg')
    if row['Negative for Pneumonia']:
        shutil.copy(row.path, f'{NP_DIR}{row.id}.jpg')
#     else:
#         print('Error: check df_train_image_level')

#     if row['Typical Appearance']:
#         shutil.copy(row.path, f'{TA_DIR}{row.id}.jpg')
#     elif row['Indeterminate Appearance']:
#         shutil.copy(row.path, f'{IA_DIR}{row.id}.jpg')
#     elif row['Atypical Appearance']:
#         shutil.copy(row.path, f'{AA_DIR}{row.id}.jpg')
#     elif row['Negative for Pneumonia']:
#         shutil.copy(row.path, f'{NP_DIR}{row.id}.jpg')
#     else:
#         print('Error: check df_train_image_level')

In [ ]:
datagen_kwargs = dict(validation_split=.20,
                      preprocessing_function=preprocess_input
                     )
dataflow_kwargs = dict(target_size=(IMG_SIZE, IMG_SIZE),
                       batch_size=BATCH_SIZE,
                       interpolation="lanczos"
                      )

In [ ]:

valid_datagen = tf.keras.preprocessing.image.ImageDataGenerator(**datagen_kwargs)
valid_generator = valid_datagen.flow_from_directory(TRAIN_DIR,
                                                    subset="validation",
                                                    shuffle=False,
                                                    **dataflow_kwargs)

train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=40,
    horizontal_flip=True,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    **datagen_kwargs)
train_generator = train_datagen.flow_from_directory(TRAIN_DIR,
                                                    subset="training",
                                                    shuffle=True,
                                                    **dataflow_kwargs)

print('classes :', train_generator.class_indices)

In [ ]:
base_model = MobileNetV2(input_shape=(224,224,3), 
                         include_top=False,
                         weights="imagenet")

In [ ]:
for layer in base_model.layers:
    layer.trainable=False

In [ ]:
model=Sequential()
model.add(base_model)
model.add(Dropout(0.5))
model.add(Flatten())
model.add(BatchNormalization())
model.add(Dense(2048,kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1024,kernel_initializer='he_uniform'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(4,activation='sigmoid'))

In [ ]:
model.summary()

In [ ]:
from tensorflow.keras.utils import plot_model
from IPython.display import SVG, Image
plot_model(model, to_file='model.png', show_shapes=True, show_layer_names=True)
Image('model.png',width=400, height=200)

In [ ]:
OPT    = tensorflow.keras.optimizers.Adam(lr=0.001)

model.compile(loss='categorical_crossentropy',
              metrics=[tensorflow.keras.metrics.Accuracy(name = 'accuracy')],
              optimizer=OPT)

In [ ]:
filepath = './best_weights.hdf5'

earlystopping = EarlyStopping(monitor = 'val_accuracy', 
                              mode = 'max' , 
                              patience = 15,
                              verbose = 1)

checkpoint    = ModelCheckpoint(filepath, 
                                monitor = 'val_accuracy', 
                                mode='max', 
                                save_best_only=True, 
                                verbose = 1)


callback_list = [earlystopping, checkpoint]

In [ ]:
model_history=model.fit(train_generator,
                        validation_data=valid_generator,
                        epochs = 30,
                        callbacks = callback_list,
                        verbose = 1)

In [ ]:
model.save('./best_weights_sigmoid.hdf5')
# model.save('./best_weights2.h5')

In [ ]:
model_path = '../input/kaggle-covid19/best_weights_sigmoid.hdf5'
model = keras.models.load_model(model_path)#'kaggle/input/kaggle-covid19/kaggle/best_weights.hdf5')
# os.listdir(model_path)

In [ ]:
model

In [ ]:
plt.plot(model_history.history['accuracy'])
plt.plot(model_history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left', bbox_to_anchor=(1,1))
plt.show()